In [4]:
import numpy as np
from ipycanvas import Canvas, hold_canvas
from math import pi, cos, sin, tan

In [2]:
from ipywidgets import FloatSlider

In [5]:
def pad_ones(x, y, z):
    return np.array([x, y, z, np.ones_like(x)])


def normalize(vec):
    return vec / np.linalg.norm(vec)


def project_vector(x, y, z, matrix):
    vec = np.dot(matrix, pad_ones(x, y, z))

    return vec[0]/vec[3], vec[1]/vec[3], vec[2]/vec[3]


def get_look_at_matrix(eye, center, up):
    n = normalize(eye - center)
    u = normalize(np.cross(up, n))
    v = np.cross(n, u)

    matrix_r = [[u[0], u[1], u[2], 0],
                [v[0], v[1], v[2], 0],
                [n[0], n[1], n[2], 0],
                [0,    0,    0,    1]]

    matrix_t = [[1, 0, 0, -eye[0]],
                [0, 1, 0, -eye[1]],
                [0, 0, 1, -eye[2]],
                [0, 0, 0, 1]]

    return np.dot(matrix_r, matrix_t)


def get_perspective_matrix(fovy, aspect, near, far):
    f = 1. / tan(fovy * pi / 360.)

    return np.array([
        [f/aspect, 0,                           0,                           0],
        [       0, f,                           0,                           0],
        [       0, 0,   (near + far)/(near - far), 2 * near * far/(near - far)],
        [       0, 0,                          -1,                           0]
    ])


class OrbitCamera():

    def __init__(self, radius, center, aspect, near=0, far=8):
        self.radius = radius
        self.center = np.array(center)
        self.aspect = aspect
        self.near = near
        self.far = far

        self.update_position(0, 0)

    def update_position(self, elev, azim):
        self.elev = elev
        self.azim = azim

        relev, razim = np.pi * self.elev/180, np.pi * self.azim/180

        xp = self.center[0] + cos(razim) * cos(relev) * self.radius
        yp = self.center[1] + sin(razim) * cos(relev) * self.radius
        zp = self.center[2] + sin(relev) * self.radius

        self.position = np.array((xp, yp, zp))
        self.front = self.center - self.position

        if abs(relev) > pi / 2.:
            self.up = np.array((0, 0, -1))
        else:
            self.up = np.array((0, 0, 1))

        self.update_matrix()

    def update_matrix(self):
        self.view_matrix = get_look_at_matrix(self.position, self.center, self.up)
        self.projection_matrix = get_perspective_matrix(50, self.aspect, self.near, self.far)
        self.matrix = np.dot(self.projection_matrix, self.view_matrix)

In [8]:
class Plot3d(Canvas):
    def __init__(self):
        super(Plot3d, self).__init__(width=500, height=500)

        self.dragging = False

        self.n = 200
        self.x = np.random.rand(self.n) - 0.5
        self.y = np.random.rand(self.n) - 0.5
        self.z = np.random.rand(self.n) - 0.5

        self.dx = 0
        self.dy = 0
        self.radius = 10

        self.camera = OrbitCamera(self.radius, [0, 0, 0], self.width / self.height)
        self.x2, self.y2, self.z2 = project_vector(
            self.x, self.y, self.z, self.camera.matrix
        )
        self.draw()

        self.on_mouse_down(self.mouse_down_handler)
        self.on_mouse_move(self.mouse_move_handler)
        self.on_mouse_up(self.mouse_up_handler)
        self.on_mouse_out(self.mouse_out_handler)

    def update_matrix(self, dx=None, dy=None, radius=None):
        dx = dx if dx is not None else self.dx
        dy = dy if dy is not None else self.dy
        self.radius = radius if radius is not None else self.radius

        self.camera.radius = self.radius
        self.camera.update_position(dy, dx)
        self.x2, self.y2, self.z2 = project_vector(
            self.x, self.y, self.z, self.camera.matrix
        )
        self.draw()

    def draw(self):
        x = self.x2 * self.width + self.width / 2
        y = self.y2 * self.height + self.height / 2
        with hold_canvas(self):
            self.clear()
            self.fill_circles(x, y, 2)

    def mouse_down_handler(self, pixel_x, pixel_y):
        self.dragging = True
        self.x_mouse = pixel_x
        self.y_mouse = pixel_y

    def mouse_move_handler(self, pixel_x, pixel_y):
        if self.dragging:
            self.dx_new = self.dx + pixel_x - self.x_mouse
            self.dy_new = self.dy + pixel_y - self.y_mouse

            self.update_matrix(self.dx_new, self.dy_new)

    def mouse_up_handler(self, pixel_x, pixel_y):
        if self.dragging:
            self.dragging = False
            self.dx = self.dx_new
            self.dy = self.dy_new

    def mouse_out_handler(self, pixel_x, pixel_y):
        if self.dragging:
            self.dragging = False
            self.dx = self.dx_new
            self.dy = self.dy_new

In [9]:
p = Plot3d()
p

Plot3d(width=500)

In [10]:
# Link Camera position to a slider widget
slider = FloatSlider(description="Radius:", min=1.0, max=7.0, value=p.radius)


def on_slider_move(change):
    slider_value = change["new"]

    p.update_matrix(radius=slider_value)


slider.observe(on_slider_move, "value")

slider

FloatSlider(value=7.0, description='Radius:', max=7.0, min=1.0)